# Automatisierte Vorverarbeitung

Das Ziel dieses Jupyter Notebook ist es die Vorverarbeitung eines rohen Datensatz automatisiert durchzufühen. Damit die einzelnen Schritte in einer Pipline abgearbeitet werden können. Die folgenden Schritte werden ausgeführt.  

1. Konkatenieren der rohen Merkmal CSV Dateien
2. Bildaufnahmen zuschneiden und skalieren
3. Überbelichtete und unterbelichtete Bilder aus dem Datensatz entfernen.
4. Messwertreihen aus der Merkmal CSV kopieren
5. Zusätzliche Merkmale aus den bestehenden berechnen 
6. Messwertreihen als eigenstädige CSV Dateien und veränderte Merkmal CSV mit den berechneten Merkmalen abspeichern als CSV Datei

In [23]:
import pandas
import os
# Regex Vergleiche
import re
import numpy as np
import cv2
import math

In [15]:
# Pfad zu den Datensatz
dataset = 52
path = "E:/bachelor/daten/" + str(dataset) + "/2018_6_22_18_38/" 
outputDir = "E:/bachelor/daten/" + str(dataset) + "/zugeschnitten/"
if not os.path.exists(outputDir):
  os.makedirs(outputDir)

In [16]:
##### 1
# Lesen der Merkmal CSV Dateien
files = os.listdir(path)
featuresDf = []
for file in files:
    if re.match('[0-9]', file) is not None:
        featuresDf.append(pandas.read_csv(filepath_or_buffer= path + file + "/merkmaleRoh.csv"))  
# Jeder Panda Dataframe wird konkateniert
length = len(featuresDf)
if length >= 1:
    result = featuresDf[0]
    for df in featuresDf[1:]:
        frames = [result, df]
        result = pandas.concat(frames, ignore_index=True)
#result.set_index('Zeitstempel in Unixzeit', inplace=True)
featuresDf = result

In [17]:
featuresDf.head()

,Zeitstempel in Unixzeit,Breitengrad,Laengengrad,Geschwindigkeit in km/h,Genauigkeit der Geschwindigkeit speedAccuracyMetersPerSecond,Lokations Zeitstempel in Unixzeit,Z-Achse Beschleunigungswerte in m/s^2,Y-Achse Beschleunigungswerte in m/s^2,Nick Messwerte in rad,Zeitstempel der Messwerte in ns,Anzahl der Messwerte,Start des Zeitfensters in ns seit Start der JVM,Zeitstempel Messwertdaten anfordern in Unixzeit,Start der Messwerterfassung in ns seit Start der JVM,Erster Zeitstempel der Teilliste in ns seit Start der JVM,Start der Belichtung in ns seit Start der JVM,Belichtungszeit in ns,Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM,Speicherzeitpunkt der Merkmale in Unixzeit
0,1532277539669,53.614082,10.162077,9.360000,0.142127,1532277540000,-2.11614 4.44431 2.56187 1.1612 -1.15422 1.504...,1.29067 -3.12904 0.12526 0.61005 1.21837 -3.25...,-0.1775 0.23919 0.00249 -0.04555 -0.13136 0.30...,5300871 13297309 21291457 29288833 37282616 45...,47,35105792553148,1532277539762,35105637941441,35105637941441,35105969862682,29996324,35106181640790,1532277540023
1,1532277540233,53.614082,10.162077,9.360000,0.142127,1532277540000,0.41175 0.06649 -0.06787 -3.24117 -0.2146 1.42...,0.83181 0.29902 3.01841 2.28686 -2.86456 0.195...,-0.08319 -0.02869 -0.31837 -0.37561 0.31717 -0...,5049276 13030245 21031059 29030102 37024979 45...,46,35106360498148,1532277540319,35105637941441,35105637941441,35106537807682,29996324,35106741336818,1532277540461
2,1532277540716,53.614105,10.162104,10.295999,0.339706,1532277541000,0.99047 -4.00051 -3.99218 -4.72935 -4.71972 1....,-1.49736 0.66902 0.66552 0.45141 0.45337 -3.51...,0.13172 -0.17447 -0.17447 -0.16139 -0.16139 0....,7552677 15547657 23544460 31539910 39533015 47...,42,35106845718660,1532277540771,35105637941441,35106181640790,35107005545682,29996324,35107197076851,1532277540876
3,1532277541103,53.614105,10.162104,10.295999,0.339706,1532277541000,3.56005 4.08806 2.30182 0.09866 -2.1955 -1.999...,-0.85307 -2.37768 0.257 -0.51544 1.06048 -1.34...,0.0642 0.17712 -0.02648 0.04768 -0.16632 0.175...,6452023 14446430 22443702 30438943 38434758 46...,42,35107246602660,1532277541171,35105637941441,35106653374950,35107406429682,29996324,35107596864572,1532277541267
4,1532277541707,53.614130,10.162135,11.987999,0.509313,1532277542000,2.49618 3.17968 2.02663 -0.44332 6.17125 1.303...,0.60865 -1.20016 -1.5657 0.23947 -2.29176 -2.0...,-0.05459 0.09377 0.13378 -0.03771 0.15159 0.19...,2951802 10949908 18940044 26938409 34932921 42...,36,35107865765700,1532277541741,35105637941441,35107053155900,35108000917682,29996324,35108164553705,1532277541817


In [5]:
##### 2
# Die Funktion schneidet und skaliert einen Datensatz von Bildern
# @param path != null / String Pfad zu den Bildern des Datensatz
#  outputDir String Pfadname des Ausgabeordners
# @return Liste mit den Pfaden zu allen zugeschnittenen Bildern
def cropAndScaleImages(path, outputDir):
    y = 85
    x = 477
    # Breite und Höhe des Ausschnits
    h = 1475
    w = 280
    # Größe des neuen Bild
    scaleFactor = 0.25
    wNew = w * scaleFactor
    hNew = h * scaleFactor
    i = -1
    filesImageNumber = []
    dirs = os.listdir(path)
    for dirNumber in dirs:
        filesImageNumber.append(os.listdir(path + dirNumber))
    for files in filesImageNumber:
        i = i + 1  
        outputDirName = outputDir + dirs[i] 
        print(outputDirName)
        if not os.path.exists(outputDirName):
            os.makedirs(outputDirName)
        for name in files:
            if "jpg" not in name:
                continue
            img = cv2.imread(path + dirs[i] + "/" + name)
            img = img[y:y+h, x:x+w]
            img = cv2.resize(img,(int(wNew),int(hNew))) # Bild wird skaliert
            cv2.imwrite( outputDirName + "/" + name, img) 
    # Liste mit den Pfaden zu den Bilder zurückgeben
    imagePaths = []
    folders = os.listdir(outputDir)
    folders = sorted(folders, key=int) #sortiert die Reihenfolge de Ordner aufsteifend
    print("Bilder aus folgenden Ordnern werden geladen: " + str(folders))
    for folder in folders: # Aus der Liste der Ordner wird ein Ordner ausgewählt
        filesPath = outputDir + folder + "/"
        files = os.listdir(filesPath)
        print("Ordner der geladen wird: " + str(folder))
        for name in files: # Ein Dateiname aus diesem Ordner
            if "jpg" not in name:
                continue
            imagePaths.append(filesPath + name) # Pfad zu den Bild wird zur Liste mit Pfaden hinzugefügt      
    return imagePaths

In [6]:
imagePaths = cropAndScaleImages(path, outputDir)

E:/bachelor/daten/52/zugeschnitten/1
E:/bachelor/daten/52/zugeschnitten/2
E:/bachelor/daten/52/zugeschnitten/3
E:/bachelor/daten/52/zugeschnitten/4
E:/bachelor/daten/52/zugeschnitten/5
Bilder aus folgenden Ordnern werden geladen: ['1', '2', '3', '4', '5']
Ordner der geladen wird: 1
Ordner der geladen wird: 2
Ordner der geladen wird: 3
Ordner der geladen wird: 4
Ordner der geladen wird: 5


In [29]:

featuresDf

,Breitengrad,Laengengrad,Geschwindigkeit in km/h,Genauigkeit der Geschwindigkeit speedAccuracyMetersPerSecond,Lokations Zeitstempel in Unixzeit,Z-Achse Beschleunigungswerte in m/s^2,Y-Achse Beschleunigungswerte in m/s^2,Nick Messwerte in rad,Zeitstempel der Messwerte in ns,Anzahl der Messwerte,...,Speicherzeitpunkt der Merkmale in Unixzeit,Mittelwert Y-Achse Beschleunigung in m/s^2,Mittelwert Z-Achse Beschleunigung in m/s^2,Mittelwert Nick in rad,Varianz Y-Achse Beschleunigung in m/s^2,Varianz Z-Achse Beschleunigung in m/s^2,Varianz Nick in rad,Standardabweichung Y-Achse Beschleunigung in m/s^2,Standardabweichung Z-Achse Beschleunigung in m/s^2,Standardabweichung Nick in rad
Zeitstempel in Unixzeit,,,,,,,,,,,,,,,,,,,,,
1532277539669,53.614082,10.162077,9.360000,0.142127,1532277540000,-2.11614 4.44431 2.56187 1.1612 -1.15422 1.504...,1.29067 -3.12904 0.12526 0.61005 1.21837 -3.25...,-0.1775 0.23919 0.00249 -0.04555 -0.13136 0.30...,5300871 13297309 21291457 29288833 37282616 45...,47,...,1532277540023,-0.104217,-0.306626,-0.065226,6.41456,14.54456,0.09951,2.53270,3.81373,0.31545
1532277540233,53.614082,10.162077,9.360000,0.142127,1532277540000,0.41175 0.06649 -0.06787 -3.24117 -0.2146 1.42...,0.83181 0.29902 3.01841 2.28686 -2.86456 0.195...,-0.08319 -0.02869 -0.31837 -0.37561 0.31717 -0...,5049276 13030245 21031059 29030102 37024979 45...,46,...,1532277540461,0.153518,-0.018573,-0.059234,4.69201,4.40261,0.06201,2.16610,2.09824,0.24902
1532277540716,53.614105,10.162104,10.295999,0.339706,1532277541000,0.99047 -4.00051 -3.99218 -4.72935 -4.71972 1....,-1.49736 0.66902 0.66552 0.45141 0.45337 -3.51...,0.13172 -0.17447 -0.17447 -0.16139 -0.16139 0....,7552677 15547657 23544460 31539910 39533015 47...,42,...,1532277540876,-0.549467,0.068882,-0.020899,7.30583,12.33093,0.08542,2.70293,3.51154,0.29227
1532277541103,53.614105,10.162104,10.295999,0.339706,1532277541000,3.56005 4.08806 2.30182 0.09866 -2.1955 -1.999...,-0.85307 -2.37768 0.257 -0.51544 1.06048 -1.34...,0.0642 0.17712 -0.02648 0.04768 -0.16632 0.175...,6452023 14446430 22443702 30438943 38434758 46...,42,...,1532277541267,-0.618322,0.376357,-0.006356,5.39327,8.07770,0.07756,2.32234,2.84213,0.27850
1532277541707,53.614130,10.162135,11.987999,0.509313,1532277542000,2.49618 3.17968 2.02663 -0.44332 6.17125 1.303...,0.60865 -1.20016 -1.5657 0.23947 -2.29176 -2.0...,-0.05459 0.09377 0.13378 -0.03771 0.15159 0.19...,2951802 10949908 18940044 26938409 34932921 42...,36,...,1532277541817,-0.391268,0.034304,0.009091,8.07712,10.72539,0.10571,2.84203,3.27496,0.32513
1532277542031,53.614130,10.162135,11.987999,0.509313,1532277542000,1.55236 1.54916 2.75036 -0.23705 -0.23662 1.15...,0.04636 0.05041 -3.64221 -0.39113 -0.39124 -2....,-0.00559 0.21762 0.29601 0.03881 0.03881 0.261...,4706315 12694941 20697006 28686152 37092070 44...,36,...,1532277542131,-0.525997,0.100932,-0.038794,7.04314,6.61421,0.09094,2.65389,2.57181,0.30156
1532277542340,53.614130,10.162135,11.987999,0.509313,1532277542000,1.4109 -4.65102 -7.71925 0.20923 2.29143 0.068...,-0.76834 -0.62757 3.19546 0.5696 -6.99278 -0.8...,0.06089 0.08269 -1.1048 -0.07724 0.54642 0.078...,7865416 15861855 23857825 31852129 39845598 47...,36,...,1532277542433,-0.413175,-0.121332,-0.044699,8.94281,17.63035,0.13404,2.99045,4.19885,0.36611
1532277542674,53.614147,10.162171,11.987999,0.538145,1532277543000,1.64981 1.64694 1.24271 1.24056 -1.8585 -0.593...,-0.28092 -0.28922 -0.39401 -0.39786 -1.00646 0...,0.02435 0.02435 0.03356 0.03356 0.12499 -0.105...,1623444 9620299 18657311 25610781 33605033 416...,36,...,1532277542754,-0.148069,0.146822,-0.011974,5.51306,4.86535,0.05970,2.34799,2.20575,0.24434
1532277542974,53.614147,10.162171,11.987999,0.538145,1532277543000,-3.50932 2.97796 2.03536 2.96792 2.2166 -0.463...,3.50949 -4.19415 -4.53775 -2.36049 -0.11957 2....,-0.56683 0.33565 0.38907 0.19575 0.01219 -0.23...,4790909 12795004 20781808 28769601 36772706 44...,36,...,1532277543046,-0.184108,0.375575,-0.037244,6.47072,6.90772,0.08096,2.54376

In [18]:
##### 3
# Berechne Indexe für Bilder die unter oder überbelichtet sind
indexe = []
imagePathsToDelete = [] # String Pfade zu den Bildern welche gelöscht werden 
i = 0
upperThreshold = 204 # obere Grauwert Schwelltwert für überbelichtete Bilder
lowerThreshold = 50  # unterer Grauwert Schwellwert für unterbelichtete Bilder
for path in imagePaths:
    img = cv2.cvtColor(cv2.imread(path),cv2.COLOR_BGR2GRAY)
    img = img.astype('float32')
    imgFlat = img.ravel()
    mean = imgFlat.sum() / imgFlat.size
    if mean > upperThreshold or mean < lowerThreshold:
        indexe.append(i)
        imagePathsToDelete.append(imagePaths[i])
    i = i + 1
# merkmaleRoh.csv Zeilen mit gefundenen Index löschen
featuresDf.drop(indexe, inplace=True)
# Die Inecies müssen zurückgesetzt werden nach den Aufruf von drop. Ansonsten kommte es zu KeyValue Fehlern
featuresDf.reset_index(drop=True,inplace=True)

In [19]:
for img in imagePathsToDelete:
    print(img)
    os.remove(img)
imagePathsToDelete.clear()

E:/bachelor/daten/52/zugeschnitten/1/1532277707843.jpg
E:/bachelor/daten/52/zugeschnitten/1/1532277872591.jpg
E:/bachelor/daten/52/zugeschnitten/1/1532278061705.jpg
E:/bachelor/daten/52/zugeschnitten/2/1532278686327.jpg
E:/bachelor/daten/52/zugeschnitten/2/1532278739585.jpg
E:/bachelor/daten/52/zugeschnitten/3/1532279206020.jpg
E:/bachelor/daten/52/zugeschnitten/3/1532279455751.jpg
E:/bachelor/daten/52/zugeschnitten/3/1532279457301.jpg
E:/bachelor/daten/52/zugeschnitten/4/1532279781620.jpg
E:/bachelor/daten/52/zugeschnitten/4/1532280233752.jpg
E:/bachelor/daten/52/zugeschnitten/4/1532280262144.jpg


In [20]:
##### 4
accYList = featuresDf['Y-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(np.array(accYList[0]).astype(float)).transpose())
for x in accYList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
accYDf = pandas.concat(accList, ignore_index=True)
# Konvertiere Beschleunigungssensordaten Z-Achse zu einem DataFrame
measuredDataList = featuresDf['Z-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    accList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
accZDf = pandas.concat(accList, ignore_index=True)
# Konvertiere Nick Messwerte zu DataFrame
measuredDataList = featuresDf['Nick Messwerte in rad'].str.split(" ").tolist()
pitchList = []
pitchList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    pitchList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
nickDf = pandas.concat(pitchList, ignore_index=True)
measuredDataList = featuresDf['Zeitstempel der Messwerte in ns'].str.split(" ").tolist()
timestampsList = []
timestampsList.append(pandas.DataFrame(np.array(measuredDataList[0]).astype(float)).transpose())
for x in measuredDataList[1:]:
    timestampsList.append(pandas.DataFrame(np.array(x).astype(float)).transpose())
sensorTimestampsDf = pandas.concat(timestampsList, ignore_index=True)

In [21]:
##### 5
def calcVariance(meansDf, dfValues):
    variance = []
    for i, values in dfValues.iterrows():
        sum = 0
        for value in values:
            if math.isnan(value) == False:
                tempDifference = value-meansDf[i]
                sum += tempDifference * tempDifference
        variance.append("{0:.5f}".format(round(sum / values.count(),5)))
    return np.array(variance).astype(float)
def calculateStandardDeviation(varianceDf):
    deviation = []
    for v in varianceDf:
        temp = math.sqrt(np.abs(v))
        deviation.append("{0:.5f}".format(round(temp,5)))
    return np.array(deviation).astype(float)

In [24]:
# Neue Spalten werden erstellt und Mittelwerte zugewiesen
featuresDf['Mittelwert Y-Achse Beschleunigung in m/s^2'] = accYDf.T.mean()
featuresDf['Mittelwert Z-Achse Beschleunigung in m/s^2'] = accZDf.T.mean()
featuresDf['Mittelwert Nick in rad'] = nickDf.T.mean()
# Berechnung der Varianz und hinzufügen der Ergebnisse als neue Spalte in dem DataFrame 
featuresDf['Varianz Y-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert Y-Achse Beschleunigung in m/s^2'],accYDf)
featuresDf['Varianz Z-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert Z-Achse Beschleunigung in m/s^2'],accZDf)
featuresDf['Varianz Nick in rad'] = calcVariance(featuresDf['Mittelwert Nick in rad'],nickDf)
# Berechne Standardabweichung
featuresDf['Standardabweichung Y-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz Y-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Z-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz Z-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Nick in rad'] = calculateStandardDeviation(featuresDf['Varianz Nick in rad'])

In [ ]:
##### 6
# Änderungen in CSV Datei schreiben
outputFolder = '../daten/merkmale_datensatz_' + str(dataset)
if not os.path.exists(outputFolder):
  os.makedirs(outputFolder)
featuresDf.set_index('Zeitstempel in Unixzeit', inplace=True)
featuresDf.to_csv('../daten/merkmale_datensatz_'+ str(dataset) + '/merkmale.csv')
# Speichert die DatenFrames mit den Messwerten in csv Dateien
accYDf.index.names = ["index"]
accYDf.to_csv('../daten/merkmale_datensatz_' + str(dataset) + '/yAchseBeschleunigungswerte.csv')
accZDf.index.names = ["index"]
accZDf.to_csv('../daten/merkmale_datensatz_' + str(dataset) + '/zAchseBeschleunigungswerte.csv')
nickDf.index.names = ["index"]
nickDf.to_csv('../daten/merkmale_datensatz_' + str(dataset) + '/nickMesswerte.csv')
sensorTimestampsDf.index.names = ["index"]
sensorTimestampsDf.to_csv('../daten/merkmale_datensatz_' + str(dataset) + '/sensoreventZeitstempel.csv')